# Student RAG Project - Guided Implementation

## Welcome!

In this project, you'll build a **RAG (Retrieval-Augmented Generation)** system that can answer questions about your documents.

### What You'll Learn:
- ✅ File I/O (reading documents)
- ✅ String manipulation (text chunking)
- ✅ Functions and parameters
- ✅ Lists and dictionaries
- ✅ Loops and conditionals
- ✅ Basic calculations and statistics

### What's Provided for You:
- ✅ Embedding model (converts text to numbers)
- ✅ Vector database (stores and searches embeddings)
- ✅ LLM connection (generates answers)

### Your Tasks:
You'll complete **TODO sections** marked with `# TODO:` comments.

Let's get started! 🚀

---
## Setup: Import Libraries

In [1]:
# Import the pre-built helper module
from rag_helpers import (
    EmbeddingModel,
    VectorDatabase,
    LLM,
    Timer,
    print_separator,
    print_search_results,
    print_rag_answer,
    check_setup
)

# Import standard Python libraries you'll use
from pathlib import Path
from typing import List, Dict
import json

# Check if everything is installed correctly
check_setup()

Checking setup...
✓ chromadb is installed
✓ sentence_transformers is installed
✓ requests is installed

✓ All required packages are installed!
You're ready to start!


True

---
## Configuration

Set up the basic settings for your RAG system.

In [2]:
# TODO: Change this to point to YOUR documents folder
DOCS_FOLDER = Path("./my_docs")

# Chunking settings (you can experiment with these!)
CHUNK_SIZE = 1000      # How many characters per chunk
OVERLAP = 150          # How many characters overlap between chunks

# How many results to retrieve for each query
TOP_K = 3

print(f"Configuration:")
print(f"  Documents folder: {DOCS_FOLDER}")
print(f"  Chunk size: {CHUNK_SIZE} characters")
print(f"  Overlap: {OVERLAP} characters")
print(f"  Top-K results: {TOP_K}")

# sanity check to ensure our files are detected
print("\nChecking files in documents folder:")
for file in DOCS_FOLDER.iterdir():
    print(" -", file.name)

Configuration:
  Documents folder: my_docs
  Chunk size: 1000 characters
  Overlap: 150 characters
  Top-K results: 3

Checking files in documents folder:
 - Document 1 - Match Duration and Basic Structure.txt
 - Document 10 - Restarts, Drop Balls, and Advantage Rule.txt
 - Document 2 - Offside Rule.txt
 - Document 3 - Fouls and Misconduct.txt
 - Document 4 - Free Kicks and Penalty Kicks.txt
 - Document 5 - Throw-Ins, Goal Kicks, and Corner Kicks.txt
 - Document 6 - Yellow and Red Cards.txt
 - Document 7 - Goalkeeper Rules and Handling Restrictions.txt
 - Document 8 - Substitutions and Player Roles.txt
 - Document 9 - Field Layout and Equipment Requirements.txt


---
## TODO #1: Document Loading

**Your Task:** Write a function to load all text files from a folder.

**What to do:**
1. Loop through all `.txt` files in the folder
2. Read each file's content
3. Store the content and filename in a dictionary
4. Return a list of these dictionaries

**Python concepts:** File I/O, loops, dictionaries, lists

In [3]:
def load_documents(folder_path: str) -> List[Dict[str, str]]:
    """
    Load all text documents from a folder.

    Args:
        folder_path: Path to folder containing .txt files

    Returns:
        List of dictionaries, each containing:
        - 'content': the text content of the file
        - 'filename': the name of the file

    Example:
        [
            {'content': 'This is doc 1...', 'filename': 'doc1.txt'},
            {'content': 'This is doc 2...', 'filename': 'doc2.txt'}
        ]
    """

    # TODO: Implement this function!
    # HINTS:
    # 1. Create an empty list to store documents
    # 2. Use Path(folder_path).glob("*.txt") to find all .txt files
    # 3. For each file:
    #    - Open it with open(file_path, 'r', encoding='utf-8')
    #    - Read the content with .read()
    #    - Create a dictionary with 'content' and 'filename'
    #    - Append to your list
    # 4. Return the list

    documents = []  # Start with empty list

    # Your code here:
    folder = Path(folder_path)

    for file_path in folder.glob("*.txt"):
        # Open and read the file
        with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
            content = f.read()
        
        # Create a dictionary
        doc_dict = {
            "content": content,
            "filename": file_path.name
        }

        # Add to documents list
        documents.append(doc_dict)
    
    print(f"✓ Loaded {len(documents)} documents")
    return documents


# Test your function
documents = load_documents(DOCS_FOLDER)

# Display first document (if any were loaded)
if documents:
    print(f"\nFirst document: {documents[0]['filename']}")
    print(f"Content preview: {documents[0]['content'][:200]}...")
else:
    print("⚠️  No documents loaded! Check your folder path.")

✓ Loaded 10 documents

First document: Document 1 - Match Duration and Basic Structure.txt
Content preview: Match Duration and Basic Structure
Soccer is played in two halves, and each half lasts 45 minutes in most standard competitions. Together, the full match is 90 minutes of regular time. After the first...


---
## TODO #2: Text Chunking Function

**Your Task:** Write a function to split long text into smaller chunks with overlap.

**Why?** Long documents are too big for embeddings. We need to split them into smaller pieces.

**What to do:**
1. Start at the beginning of the text
2. Take a chunk of `chunk_size` characters
3. Move forward by `chunk_size - overlap` characters
4. Repeat until you reach the end

**Python concepts:** String slicing, loops, lists

In [4]:
def chunk_text(text: str, chunk_size: int = 500, overlap: int = 50) -> List[str]:
    """
    Split text into overlapping chunks.

    Args:
        text: The text to split
        chunk_size: Maximum characters per chunk
        overlap: How many characters to overlap between chunks

    Returns:
        List of text chunks

    Example:
        text = "This is a long document..."
        chunks = chunk_text(text, chunk_size=100, overlap=20)
        # Returns: ['This is a long...', 'long document...']
    """

    # TODO: Implement this function!
    # HINTS:
    # 1. Create an empty list to store chunks
    # 2. Start with position = 0
    # 3. While position < len(text):
    #    - Extract chunk from position to position+chunk_size
    #    - Add chunk to list (if not empty)
    #    - Move position forward by (chunk_size - overlap)
    # 4. Return the list of chunks

    chunks = []  # Start with empty list
    position = 0  # Start at beginning

    # Your code here:
    while position < len(text):
        # Extract a chunk
        chunk = text[position : position + chunk_size]

        # Add to chunks list
        if chunk.strip():
            chunks.append(chunk)

        # Move position forward
        position += chunk_size - overlap

    return chunks


# Test your chunking function
test_text = "This is a test. " * 50  # Create a long test string
test_chunks = chunk_text(test_text, chunk_size=100, overlap=20)

print(f"Test text length: {len(test_text)} characters")
print(f"Number of chunks: {len(test_chunks)}")
print(f"\nFirst chunk: {test_chunks[0]}")
if len(test_chunks) > 1:
    print(f"Second chunk: {test_chunks[1]}")

Test text length: 800 characters
Number of chunks: 10

First chunk: This is a test. This is a test. This is a test. This is a test. This is a test. This is a test. This
Second chunk: This is a test. This is a test. This is a test. This is a test. This is a test. This is a test. This


---
## TODO #3: Process All Documents into Chunks

**Your Task:** Use your chunking function to split ALL documents into chunks and create metadata.

**What to do:**
1. Loop through each document
2. Chunk the document's content
3. For each chunk, create metadata (which file it came from, which chunk number)
4. Store everything in a list

**Python concepts:** Nested loops, dictionaries, enumerate

In [5]:
def process_documents(documents: List[Dict[str, str]],
                     chunk_size: int = 500,
                     overlap: int = 50) -> tuple:
    """
    Process all documents into chunks with metadata.

    Args:
        documents: List of document dictionaries
        chunk_size: Size of each chunk
        overlap: Overlap between chunks

    Returns:
        Tuple of (chunk_texts, chunk_metadatas)
        - chunk_texts: List of chunk strings
        - chunk_metadatas: List of metadata dictionaries
    """

    # TODO: Implement this function!
    # HINTS:
    # 1. Create two empty lists: chunk_texts and chunk_metadatas
    # 2. For each document:
    #    - Get the document's content and filename
    #    - Use your chunk_text() function to split it
    #    - For each chunk (use enumerate to get index):
    #      - Add chunk text to chunk_texts
    #      - Create metadata dict with 'source' and 'chunk_id'
    #      - Add metadata to chunk_metadatas
    # 3. Return both lists as a tuple

    chunk_texts = []
    chunk_metadatas = []

    # Your code here:
    for doc in documents:
        # Get document content and filename
        content = doc["content"]
        filename = doc["filename"]

        # Chunk the document
        chunks = chunk_text(content, chunk_size=chunk_size, overlap=overlap)

        # For each chunk:
        for idx, chunk in enumerate(chunks):
            #   - Add chunk text to chunk_texts
            chunk_texts.append(chunk)
            #   - Create metadata dictionary
            metadata = {
                "source": filename,
                "chunk_id": idx
            }

            #   - Add metadata to chunk_metadatas
            chunk_metadatas.append(metadata)


    print(f"✓ Created {len(chunk_texts)} chunks from {len(documents)} documents")
    return chunk_texts, chunk_metadatas


# Process all documents
chunk_texts, chunk_metadatas = process_documents(documents, CHUNK_SIZE, OVERLAP)

# Display example
if chunk_texts:
    print(f"\nExample chunk:")
    print(f"  Source: {chunk_metadatas[0]['source']}")
    print(f"  Chunk ID: {chunk_metadatas[0]['chunk_id']}")
    print(f"  Text: {chunk_texts[0][:200]}...")

✓ Created 20 chunks from 10 documents

Example chunk:
  Source: Document 1 - Match Duration and Basic Structure.txt
  Chunk ID: 0
  Text: Match Duration and Basic Structure
Soccer is played in two halves, and each half lasts 45 minutes in most standard competitions. Together, the full match is 90 minutes of regular time. After the first...


---
## Pre-Built: Create Embeddings and Store in Database

This part uses the pre-built helpers. Just run these cells - no coding needed! ✨

In [6]:
# Initialize the embedding model (pre-built)
print("Initializing embedding model...")
embedder = EmbeddingModel()

# Create embeddings for all chunks (pre-built)
print("\nCreating embeddings...")
embeddings = embedder.embed_multiple(chunk_texts)
print(f"✓ Created {len(embeddings)} embeddings")

Initializing embedding model...
Loading embedding model: sentence-transformers/all-MiniLM-L6-v2
✓ Model loaded!

Creating embeddings...
Embedding 20 texts...
✓ Complete!
✓ Created 20 embeddings


In [7]:
# Initialize vector database (pre-built)
print("Initializing vector database...")
vector_db = VectorDatabase()

# Add chunks to database (pre-built)
print("\nAdding chunks to database...")
vector_db.add_chunks(chunk_texts, embeddings, chunk_metadatas)

Initializing vector database...
✓ Vector database initialized
  Collection: student_rag
  Current documents: 256

Adding chunks to database...
✓ Added 20 chunks to database


In [8]:
# Initialize LLM connection (pre-built)
print("Connecting to Ollama LLM...")
llm = LLM()


# Test the connection
print("\nTesting LLM connection...")
if llm.test_connection():
    print("✓ LLM is working!")
else:
    print("⚠️  LLM connection failed! Make sure Docker container is running.")


Connecting to Ollama LLM...
✓ LLM initialized: mistral at http://127.0.0.1:11434

Testing LLM connection...
✓ LLM is working!


---
## TODO #4: RAG Query Function

**Your Task:** Write the main RAG function that ties everything together!

**What to do:**
1. Embed the user's question
2. Search the database for similar chunks
3. Build a prompt with the retrieved context
4. Ask the LLM to answer based on the context
5. Return the answer and metadata

**Python concepts:** Functions, string formatting, dictionaries

In [9]:
def rag_query(question: str, top_k: int = 3) -> Dict:
    """
    Answer a question using RAG (Retrieval-Augmented Generation).

    Args:
        question: The user's question
        top_k: How many chunks to retrieve

    Returns:
        Dictionary with:
        - 'question': the original question
        - 'answer': the LLM's answer
        - 'sources': list of source filenames
        - 'contexts': list of retrieved chunks
        - 'time': how long it took
    """

    # Start timer
    timer = Timer()
    timer.start()

    # TODO: Implement the RAG pipeline!
    # HINTS:
    # 1. Embed the question using: embedder.embed_text(question)
    # 2. Search database using: vector_db.search(query_embedding, top_k)
    # 3. Extract retrieved chunks and metadata from search results:
    #    - retrieved_chunks = results['documents'][0]
    #    - retrieved_metadata = results['metadatas'][0]
    # 4. Build context by joining chunks with newlines
    # 5. Create prompt (template below)
    # 6. Generate answer using: llm.generate_answer(prompt)
    # 7. Extract source filenames from metadata
    # 8. Return everything in a dictionary

    # Step 1: Embed question
    query_embedding = embedder.embed_text(question)

    # Step 2: Search database
    results = vector_db.search(query_embedding, top_k)

    # Step 3: Extract results
    retrieved_chunks = results["documents"][0]
    retrieved_metadata = results["metadatas"][0]

    # Step 4: Build context
    context = "\n\n".join(retrieved_chunks)

    # Step 5: Create prompt (use this template)
    prompt = f"""Answer the question based on the context below.

Context:
{context}

Question: {question}

Answer:"""

    # Step 6: Generate answer
    answer = llm.generate_answer(prompt)

    # Step 7: Extract sources
    sources = [meta["source"] for meta in retrieved_metadata]

    # Stop timer
    elapsed_time = timer.stop()

    # Step 8: Return results
    return {
        'question': question,
        'answer': answer,
        'sources': sources,
        'contexts': retrieved_chunks,
        'time': elapsed_time
    }


print("✓ RAG query function defined!")

✓ RAG query function defined!


---
## Test Your RAG System!

Let's try asking some questions!

In [15]:
# Test question 1
result = rag_query(" Please tell me who is the best player? Ronaldo or Messi?")

# Pretty print the result
print_rag_answer(
    result['question'],
    result['answer'],
    result['sources'],
    result['time']
)

======================== RAG ANSWER ========================

QUESTION:  Please tell me who is the best player? Ronaldo or Messi?

ANSWER:
 It's subjective to determine who the "best" player between Cristiano Ronaldo and Lionel Messi is, as their performances and achievements vary over time. Both have won multiple individual awards and consistently perform at a high level in international football and for their respective clubs. The choice often depends on personal preferences based on factors such as style of play, team success, or specific accomplishments.

SOURCES: Document 8 - Substitutions and Player Roles.txt
TIME: 8.94 seconds


In [ ]:
# Try your own question!
my_question = "What conditions must be met for a player to be offside?"  

result = rag_query(my_question)
print_rag_answer(
    result['question'],
    result['answer'],
    result['sources'],
    result['time']
)

---
## TODO #5: Create Test Dataset

**Your Task:** Create a list of test questions to evaluate your RAG system.

**What to do:**
1. Think of 10 questions your documents can answer
2. For each question, write the expected answer
3. Store them in a structured format

**Python concepts:** Lists, dictionaries, data structures

In [ ]:
# TODO: Create your test dataset!
# HINTS:
# Create a list of dictionaries
# Each dictionary should have:
#   - 'question': the test question
#   - 'expected_answer': what you expect the answer to include
#   - 'category': type of question (factual, inferential, etc.)

test_questions = [
    # Example (replace with your own!):
    # ---------- FACTUAL (4) ----------
    {
        'question': 'How long is a standard soccer match and what is the purpose of stoppage time?',
        'expected_answer': 'A standard soccer match lasts 90 minutes, divided into two 45-minute halves. Stoppage time compensates for interruptions such as injuries, substitutions, and delays.',
        'category': 'factual'
    },
    {
        'question': 'What conditions must be met for a player to be offside?',
        'expected_answer': 'A player is offside if they are ahead of both the ball and the second-last defender in the opponents’ half at the moment the ball is played to them.',
        'category': 'factual'
    },
    {
        'question': 'Do only players receive yellow or red cards?',
        'expected_answer': 'No, in addition to players, team officials such as coaches or staff can also receive yellow or red cards for inappropriate behavior during a match.',
        'category': 'factual'
    },
    {
        'question': 'How many substitutions are typically allowed in a match?',
        'expected_answer': 'Most competitions allow three to five substitutions depending on the league or tournament rules.',
        'category': 'factual_basic'
    },

    # ---------- INFERENTIAL (3) ----------
    {
        'question': 'Why is receiving a red card early in the match a major disadvantage for a team?',
        'expected_answer': 'A red card forces a team to continue with one fewer player, reducing defensive stability and limiting attacking opportunities for the rest of the match.',
        'category': 'inferential'
    },
    {
        'question': 'If a defender commits a careless foul outside the box, what restart is awarded and why?',
        'expected_answer': 'A direct free kick is awarded because the foul occurred outside the penalty area and was judged careless, giving the attacking team a chance to restart play.',
        'category': 'inferential'
    },
    {
        'question': 'Why might a team choose to commit a tactical foul?',
        'expected_answer': 'Teams commit tactical fouls to stop counterattacks, break the opponent’s momentum, or regain defensive shape when vulnerable.',
        'category': 'inferential'
    },

    # ---------- DEEP_KNOWLEDGE (2) ----------
    {
        'question': 'How does a referee keep control of the game?',
        'expected_answer': 'Referees maintain control by enforcing the rules, issuing cautions or cards when needed, using clear communication, and managing player behavior.',
        'category': 'deep_knowledge'
    },
    {
        'question': 'What usually happens if a team keeps committing minor fouls?',
        'expected_answer': 'Repeated minor fouls often lead to warnings, yellow cards, or loss of match control for the offending team.',
        'category': 'deep_knowledge'
    },

    # ---------- FUNNY (3) ----------
    {
        'question': 'What is Cristiano Ronaldo’s favorite food?',
        'expected_answer': 'The documents do not contain information about Cristiano Ronaldo’s favorite food.',
        'category': 'funny'
    },
    {
        'question': 'Can a goalkeeper drink tea during the match while waiting for the ball?',
        'expected_answer': 'The documents do not mention tea drinking, but goalkeepers must stay alert and ready, so it would not be practical.',
        'category': 'funny'
    },
    {
        'question': 'If a player celebrates by breakdancing on the field, is it allowed?',
        'expected_answer': 'The documents do not specifically mention breakdancing, but unsafe or excessive celebrations may lead to a caution.',
        'category': 'funny'
    },

    # ---------- EDGE_CASES (5) ----------
    {
        'question': 'What happens if both teams commit fouls at the same time?',
        'expected_answer': 'The referee determines which foul occurred first or which offense had greater impact, then restarts play accordingly.',
        'category': 'edge_cases'
    },
    {
        'question': 'What happens if a player wears a formal suit during the match?',
        'expected_answer': 'Formal clothing does not meet equipment requirements, so the player would be asked to change before participating.',
        'category': 'edge_cases'
    },
    {
        'question': 'What happens if one of the referees gets injured?',
        'expected_answer': 'A substitute referee or another official typically replaces the injured referee depending on match regulations.',
        'category': 'edge_cases'
    },
    {
        'question': 'Is it legal to score directly from a throw-in?',
        'expected_answer': 'No, a goal cannot be scored directly from a throw-in according to the rules.',
        'category': 'edge_cases'
    },
    {
        'question': 'Can the goalkeeper receive a red card just for touching the ball?',
        'expected_answer': 'No, touching the ball alone is not a red card offense; a red card requires a serious violation such as violent conduct.',
        'category': 'edge_cases'
    },

    # ---------- Tactical/ Strategy (3) ----------
    {
        'question': 'Why might a team target the wings instead of the center?',
        'expected_answer': 'The wings often provide more attacking space, fewer defenders, and better opportunities for crossing into the box.',
        'category': 'tactical_strategy'
    },
    {
        'question': 'How does a high defensive line affect offside calls?',
        'expected_answer': 'A high defensive line increases the likelihood of attackers being caught offside because defenders play closer to midfield.',
        'category': 'tactical_strategy'
    },
    {
        'question': 'Why might a team commit intentional fouls in midfield?',
        'expected_answer': 'Intentional midfield fouls slow down attacks, disrupt the opponent’s rhythm, and allow the defense to reset.',
        'category': 'tactical_strategy'
    },

    # ---------- OUT_OF_DOMAIN (3) ----------
    {
        'question': 'How many galaxies exist in the observable universe?',
        'expected_answer': 'The documents only cover soccer rules, so they do not contain information about galaxies or astronomy.',
        'category': 'out_of_domain'
    },
    {
        'question': 'Who invented the first electric piano?',
        'expected_answer': 'The documents include soccer rule information only and do not cover musical instrument history.',
        'category': 'out_of_domain'
    },
    {
        'question': 'What is the boiling point of liquid nitrogen?',
        'expected_answer': 'The documents do not discuss scientific measurements such as the boiling point of liquid nitrogen.',
        'category': 'out_of_domain'
    },

    # ---------- HYPOTHETICAL_SCENARIOS (2) ----------
    {
        'question': 'How would the offside rule apply if soccer was played with flying drones?',
        'expected_answer': 'The offside rule is designed for human players; drone soccer would require new rules based on drone movement and positioning.',
        'category': 'hypothetical_scenarios'
    },
    {
        'question': 'If soccer had three goals instead of two, how would restarts change?',
        'expected_answer': 'The documents do not describe three-goal soccer, but restart rules would likely need adjustments to accommodate a different field layout.',
        'category': 'hypothetical_scenarios'
    }   
]

print(f"✓ Created {len(test_questions)} test questions")
print(f"\nExample question:")
print(f"  Q: {test_questions[0]['question']}")
print(f"  Expected: {test_questions[0]['expected_answer']}")

---
## TODO #6: Calculate Evaluation Metrics

**Your Task:** Write functions to measure how well your RAG system performs.

**Python concepts:** Functions, calculations, statistics

In [ ]:
def calculate_average_latency(results: List[Dict]) -> float:
    """
    Calculate average response time.

    Args:
        results: List of result dictionaries (each has 'time' field)

    Returns:
        Average time in seconds
    """

    # TODO: Implement this function!
    # HINTS:
    # 1. Extract all 'time' values from results
    # 2. Sum them up
    # 3. Divide by the number of results
    # 4. Return the average

    # Your code here:
    total_time = sum(result['time'] for result in results)

    # Calculate sum and average
    avg_time = total_time / len(results) if results else 0.0


    return avg_time 


def count_successful_retrievals(results: List[Dict]) -> int:
    """
    Count how many queries successfully retrieved context.

    Args:
        results: List of result dictionaries

    Returns:
        Number of successful retrievals
    """

    # TODO: Implement this function!
    # HINTS:
    # 1. Start with count = 0
    # 2. For each result:
    #    - Check if 'contexts' is not empty
    #    - If yes, increment count
    # 3. Return count

    # Your code here:
    count = 0
    # Count successful retrievals
    for result in results:
        # Successful if 'contexts' is NOT empty
        if result['contexts']:
            count += 1

    return count


def get_all_sources(results: List[Dict]) -> List[str]:
    """
    Get unique list of all sources used.

    Args:
        results: List of result dictionaries

    Returns:
        List of unique source filenames
    """

    # TODO: Implement this function!
    # HINTS:
    # 1. Create an empty set (sets automatically keep unique values)
    # 2. For each result:
    #    - Get the 'sources' list
    #    - Add each source to the set
    # 3. Convert set to list and return

    # Your code here:
    all_sources = set()

    # Collect all sources
    for result in results:
        # result['sources'] is a list → add each
        for src in result['sources']:
            all_sources.add(src)

    return list(all_sources)


print("✓ Evaluation functions defined!")

---
## TODO #7: Run Complete Evaluation

**Your Task:** Test your RAG system with all test questions and calculate metrics.

**Python concepts:** Loops, function calls, data aggregation

In [ ]:
def run_evaluation(test_questions: List[Dict]) -> List[Dict]:
    """
    Run RAG system on all test questions.

    Args:
        test_questions: List of test question dictionaries

    Returns:
        List of result dictionaries
    """

    # TODO: Implement this function!
    # HINTS:
    # 1. Create empty list for results
    # 2. For each test question:
    #    - Get the question text
    #    - Call rag_query() with the question
    #    - Add result to results list
    # 3. Return results

    results = []

    # Your code here:
    for test in test_questions:
        # Get question
        question = test['question']
        # Run RAG query
        result = rag_query(question, top_k=TOP_K)

        # Attach expected_answer and category to the result
        result['expected_answer'] = test.get('expected_answer', '')
        result['category'] = test.get('category', 'unknown')

        # Store result
        results.append(result)

    return results


# Run evaluation on all test questions
print("Running evaluation on all test questions...\n")
all_results = run_evaluation(test_questions)

print(f"\n✓ Completed {len(all_results)} tests")

---
## Display Results

Show the evaluation metrics and results.

In [ ]:
# Calculate metrics using your functions
avg_latency = calculate_average_latency(all_results)
successful_retrievals = count_successful_retrievals(all_results)
all_sources_used = get_all_sources(all_results)
hit_rate = successful_retrievals / len(all_results) if all_results else 0

# Display metrics
print_separator("EVALUATION RESULTS")
print(f"\nTotal Questions Tested: {len(all_results)}")
print(f"Successful Retrievals: {successful_retrievals}")
print(f"Hit Rate: {hit_rate:.2%}")
print(f"Average Latency: {avg_latency:.2f} seconds")
print(f"\nSources Used: {', '.join(all_sources_used)}")
print_separator()

In [ ]:
# Display individual results
print("\nIndividual Test Results:\n")

for i, result in enumerate(all_results, 1):
    print(f"[Test {i}]")
    print(f"Question: {result['question']}")
    print(f"Answer: {result['answer'][:200]}...")
    print(f"Sources: {', '.join(set(result['sources']))}")
    print(f"Time: {result['time']:.2f}s")
    print("-" * 60)
    print()

---
## Save Your Results

Save your test results to a JSON file for your report.

In [ ]:
# Save results to JSON file
results_summary = {
    'metrics': {
        'total_questions': len(all_results),
        'successful_retrievals': successful_retrievals,
        'hit_rate': hit_rate,
        'average_latency': avg_latency
    },
    'results': all_results
}

with open('evaluation_results.json', 'w') as f:
    json.dump(results_summary, f, indent=2)

print("✓ Results saved to 'evaluation_results.json'")

---
## Congratulations! 🎉

You've successfully built a RAG system!

### What You Accomplished:
✅ Loaded documents from files  
✅ Chunked text with overlap  
✅ Created a RAG query pipeline  
✅ Built a test dataset  
✅ Calculated evaluation metrics  
✅ Generated a results report  

### Next Steps:
- Try different chunk sizes and overlaps
- Add more test questions
- Experiment with different values for `top_k`
- Analyze which questions work best
- Write up your findings in a report

### For Your Report:
1. Describe your document collection
2. Explain your chunking strategy
3. Present your evaluation metrics
4. Show examples of good and bad answers
5. Discuss what you learned

Great job! 🚀